In [1]:
# Initial Setup and Imports
import sys
import os

# Get the current working directory and navigate to project root
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# From schema folder, go up one level to reach project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Verify the utils directory exists
utils_path = os.path.join(project_root, 'utils')
print(f"Utils path exists: {os.path.exists(utils_path)}")

try:
    from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer
    print("Successfully imported utils functions!")
except ImportError as e:
    print(f"Import error: {e}")
    print(f"sys.path: {sys.path}")

client, model_name, api_provider = setup_llm_client(model_name="o3")

convisoft_prd = load_artifact("artifacts/convisoft_prd.md")

Current working directory: c:\Users\labadmin\Desktop\Labs\KMSH_contracting\schema
Project root: c:\Users\labadmin\Desktop\Labs\KMSH_contracting
Added c:\Users\labadmin\Desktop\Labs\KMSH_contracting to sys.path
Utils path exists: True
Successfully imported utils functions!


2025-10-03 13:29:02,259 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


In [5]:
# Generate the Pydantic Models for Data Validation
schema_prompt = prompt_enhancer(f"""
You are an expert database architect. Given the following Product Requirements Document (PRD), generate a complete SQL schema for a relational database that fulfills the requirements. Ensure to include tables, columns with appropriate data types, primary keys, foreign keys, and any necessary constraints.
The database should use standard SQL syntax compatible with SQLite.
Create any join tables needed for many-to-many relationships.

PRD:
{convisoft_prd}
""")

print("--- Generating SQL Schema ---")
if convisoft_prd:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'capstone/convisoft_schema.sql', overwrite=True)

2025-10-02 11:07:12,582 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating SQL Schema ---
-- Enable foreign-key enforcement in SQLite
PRAGMA foreign_keys = ON;

--------------------------------------------------------------------
-- 1. Reference / Lookup Tables
--------------------------------------------------------------------
CREATE TABLE naics_codes (
    naics_code        TEXT PRIMARY KEY,               -- 6-digit string
    description       TEXT NOT NULL
);

CREATE TABLE psc_codes (
    psc_code          TEXT PRIMARY KEY,               -- 4-character string
    description       TEXT NOT NULL
);

CREATE TABLE roles (
    role_id           INTEGER PRIMARY KEY,
    role_name         TEXT NOT NULL UNIQUE,           -- e.g., admin, analyst
    description       TEXT
);

--------------------------------------------------------------------
-- 2. Core Master Tables
--------------------------------------------------------------------
CREATE TABLE locations (
    location_id       INTEGER PRIMARY KEY,
    address_line1     TEXT NOT NULL,
    addr

In [7]:
convisoft_schema = load_artifact("capstone/convisoft_schema.sql")
db_code_prompt = f"""
You're a Senior Python Developer tasked with creating SQLAlchemy models.
Using the SQL schema below, generate the following:
    SQLAlchemy Models: For each table in the schema, create a corresponding SQLAlchemy model class.

SQL: {convisoft_schema}
"""

print("--- Generating SQLAlchemy Models ---")
if convisoft_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
    save_artifact(generated_db_code, 'capstone/convisoft_db.py', overwrite=True)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models ---

--- Generated Database Code ---
```python
"""
SQLAlchemy ORM models that mirror the schema supplied in the prompt.

• Compatible with SQLAlchemy ≥ 2.0 (works in 1.4 with future-style engine as well)
• All FK “ON UPDATE / ON DELETE” rules present
• Default / on-update timestamps handled with func.now()
• Helpful indexes & CHECK constraints preserved
"""

from __future__ import annotations

from datetime import date, datetime

from sqlalchemy import (
    CheckConstraint,
    Column,
    Date,
    DateTime,
    Float,
    ForeignKey,
    Index,
    Integer,
    Numeric,
    String,
    Table,
    Text,
    UniqueConstraint,
    func,
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship


# --------------------------------------------------------------------------- #
#  Base class
# --------------------------------------------------------------------------- #
class Base(DeclarativeBase):
    """Declarative base for all ORM

In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Load PRD from file
convisoft_prd = load_artifact("capstone/convisoft_prd.md")

llm = ChatOpenAI(model="gpt-4o")

# Tavily search tool (LangChain community tool)
search_tool = TavilySearchResults(max_results=3)

# 1. Extract entities and relationships from PRD
extract_entities_prompt = PromptTemplate.from_template(
    """You are a senior product analyst.
Given the following PRD, extract all relevant entities, attributes, and relationships needed for a database schema.
If any information is missing or ambiguous, use web search tools to find details from referenced websites (sam.gov, gsa.gov, fpds.gov).
PRD:
{prd}
Return a structured list of entities, attributes, and relationships.

{agent_scratchpad}"""
)
extract_entities_chain = LLMChain(llm=llm, prompt=extract_entities_prompt)

# 2. Generate SQL schema from extracted requirements
schema_gen_prompt = PromptTemplate.from_template(
    """You are a database architect.
Given these requirements, generate a complete SQL schema for a relational database (SQLite syntax).
If requirements are unclear, use web search tools to clarify using referenced government sites.
Requirements:
{requirements}
Return only the SQL schema.

{agent_scratchpad}"""
)
schema_gen_chain = LLMChain(llm=llm, prompt=schema_gen_prompt)

# 3. Generate SQLAlchemy models from SQL schema
alchemy_gen_prompt = PromptTemplate.from_template(
    """You are a senior Python developer.
Given the following SQL schema, generate SQLAlchemy ORM model classes (Python code).
If best practices are unclear, use web search tools to check.
SQL Schema:
{schema}
Return only the Python code for the models.

{agent_scratchpad}"""
)
alchemy_gen_chain = LLMChain(llm=llm, prompt=alchemy_gen_prompt)

# Agent 1: Entity extraction agent
extract_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=extract_entities_prompt
    verbose=True
)
extract_executor = AgentExecutor(agent=extract_agent, tools=[search_tool], verbose=True)

# Step 1: Extract entities/relationships
entities_result = extract_executor.invoke({"prd": convisoft_prd})
entities = entities_result["output"] if "output" in entities_result else entities_result

print("\n--- Entities & Relationships ---\n")
print(entities)

# Agent 2: Schema generation agent
schema_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=schema_gen_prompt,
    verbose=True
)
schema_executor = AgentExecutor(agent=schema_agent, tools=[search_tool], verbose=True)

# Step 2: Generate SQL schema
schema_result = schema_executor.invoke({"requirements": entities})
schema = schema_result["output"] if "output" in schema_result else schema_result

print("\n--- SQL Schema ---\n")
print(schema)

# Agent 3: SQLAlchemy model generation agent
alchemy_agent = create_tool_calling_agent(
    llm=llm,
    tools=[search_tool],
    prompt=alchemy_gen_prompt,
    verbose=True
)
alchemy_executor = AgentExecutor(agent=alchemy_agent, tools=[search_tool], verbose=True)

# Step 3: Generate SQLAlchemy models
alchemy_result = alchemy_executor.invoke({"schema": schema})
alchemy_models = alchemy_result["output"] if "output" in alchemy_result else alchemy_result

print("\n--- SQLAlchemy Models ---\n")
print(alchemy_models)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (362434241.py, line 59)

In [7]:
import sys
import os
import sqlite3

# Get the current working directory and navigate to project root
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# From schema folder, go up one level to reach project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))
print(f"Project root: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Verify the utils directory exists
utils_path = os.path.join(project_root, 'utils')
print(f"Utils path exists: {os.path.exists(utils_path)}")

try:
    from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer
    print("Successfully imported utils functions!")
except ImportError as e:
    print(f"Import error: {e}")
    print(f"sys.path: {sys.path}")

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")


def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        # Drop each table
        for table_name in tables:
            table_name = table_name[0]  # Extract table name from tuple
            if table_name == "sqlite_sequence":
                continue  # Skip internal SQLite table
            cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
            print(f"Dropped table: {table_name}")
        
        # Commit changes
        conn.commit()
        print("Database cleaned successfully.")

        # TODO: Read the content of the schema file using load_artifact.
        schema_sql = load_artifact(schema_path)
        if not schema_sql:
            print(f"Error: Could not read schema from {schema_path}")
            return
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        cursor.executescript(schema_sql)
        
        print("Tables created successfully.")

         # Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            seed_sql = load_artifact(seed_path)
            if seed_sql:
                # Split the seed SQL into individual statements
                seed_statements = seed_sql.split(";")
                for i, statement in enumerate(seed_statements):
                    statement = statement.strip()
                    if statement:  # Skip empty statements
                        try:
                            cursor.execute(statement)
                        except sqlite3.Error as e:
                            print(f"Error inserting seed data at statement {i + 1}: {statement}")
                            print(f"SQLite Error: {e}")
                            break
                else:
                    print("Seed data inserted successfully.")
            else:
                print(f"Warning: Could not read seed data from {seed_path}")

        # TODO: Commit the changes to the database.
        conn.commit()
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        if conn:
            conn.close()
        print("Database connection closed.")

# Define file paths
db_file = os.path.join(project_root, "artifacts", "convisoft.db")
schema_file = os.path.join(project_root, "artifacts", "convisoft_schema.sql")
seed_file = os.path.join(project_root, "artifacts", "convisoft_seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

2025-10-03 13:33:35,470 ag_aisoftdev.utils ERROR No module named 'google' provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


Current working directory: c:\Users\labadmin\Desktop\Labs\KMSH_contracting\schema
Project root: c:\Users\labadmin\Desktop\Labs\KMSH_contracting
Utils path exists: True
Successfully imported utils functions!
Successfully connected to database at c:\Users\labadmin\Desktop\Labs\KMSH_contracting\artifacts\convisoft.db
Dropped table: naics_codes
Dropped table: psc_codes
Dropped table: roles
Dropped table: locations
Dropped table: companies
Dropped table: users
Dropped table: user_roles
Dropped table: contracts
Dropped table: contract_naics
Dropped table: contract_psc
Database cleaned successfully.
Tables created successfully.
Error inserting seed data at statement 7: --------------------------------------------------------------------
-- 6. Contracts (All Real Government Contracts from USASpending.gov)
--------------------------------------------------------------------
INSERT OR IGNORE INTO contracts (contract_id, contract_number, title, company_id, place_of_performance_location_id, date_a